In [2]:
#Shengtao Lin, Tianhe Wang

# Twitter Api

In [1]:
!pip install tweepy

In [1]:
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream

In [2]:
#key and tokens
#api key: bUvI6HoYH97pwjTmnMNAPTMTS
#api secret key: Hp420vZADZ25NMR0sqEf4eVlYXVtUsyL4AhwgdAxTf2ssbr0in
#access token: 1190435999551303680-vK990ORPT4tj7KU1TTkfOtARxNzLfz
#Access Token Secret: 6KWkz1UYuCPvvGKk22AvMumRpwlA2kO8DlIVZ5KwpQtTK

access_token = "1190435999551303680-vK990ORPT4tj7KU1TTkfOtARxNzLfz"
access_token_secret = "6KWkz1UYuCPvvGKk22AvMumRpwlA2kO8DlIVZ5KwpQtTK"
consumer_key = "bUvI6HoYH97pwjTmnMNAPTMTS"
consumer_secret = "Hp420vZADZ25NMR0sqEf4eVlYXVtUsyL4AhwgdAxTf2ssbr0in"

In [67]:
import sys
default_stdout = sys.stdout
sys.stdout = open('AI-out', 'w')

In [68]:
class StdOutListener(StreamListener):
    t_count=0
    
    def on_data(self, data):
        print (data)
        self.t_count += 1
        
        #stop by 
        if self.t_count >=12000:
            sys.exit("End")
            
        return True

    def on_error(self, status):
        print (status)
    


if __name__ == '__main__':

    #This handles Twitter authetification and the connection to Twitter Streaming API
    l = StdOutListener()
    auth = OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_token_secret)
    stream = Stream(auth, l)

    #This line filter Twitter Streams to capture data by the keywords: 'python', 'javascript', 'ruby'
    stream.filter(track=['#ArtificialIntelligence'],languages=["en"])

SystemExit: End

In [70]:
sys.stdout = default_stdout

In [30]:
import json
count=0
with open("AI-out", "r") as f1:
    for line in f1:
        count+=1
        try:
            data = json.loads(line)
            # if you want to see a specific field, you can print it. 
            #If your file is big, there may be too many of these printed
            #print(data['text']) 
        except:
            continue
print(count)

24000


# DataBase create/insert

In [3]:
#Database import
import mysql.connector
import pymongo
from datetime import datetime
import sys
import json

In [5]:
mydbsql = mysql.connector.connect(
  host="localhost",
  user="root",
  password="789456565632",
  database="TwitterDB"
)

In [6]:
mycursor = mydbsql.cursor(buffered=True)

In [7]:
#create DATABASE
mycursor.execute("CREATE DATABASE TwitterDB")

DatabaseError: 1007 (HY000): Can't create database 'twitterdb'; database exists

In [21]:
#Create Table
#mycursor.execute("USE TwitterDB")
mycursor.execute("CREATE TABLE IF NOT EXISTS user("+
                 "user_id VARCHAR(255) PRIMARY KEY,"+
                "name VARCHAR(255),"+
                "screen_name VARCHAR(255),"+
                "location VARCHAR(255),"+
                "description VARCHAR(255),"+
                "followers_count INT NOT NULL,"+
                "friends_count INT NOT NULL,"+
                "listed_count INT NOT NULL,"+
                "favourites_count INT NOT NULL,"+
                "statuses_count INT NOT NULL,"+
                "created_at DATETIME NOT NULL);")

In [22]:
mycursor.execute("CREATE TABLE IF NOT EXISTS post("+
                 "user_id VARCHAR(255) NOT NULL,"+
                 "post_id VARCHAR(255) NOT NULL,"+
                 "created_at DATETIME NOT NULL,"+
                 "PRIMARY KEY (user_id, post_id),"+
                 "FOREIGN KEY (user_id) REFERENCES user(user_id));")

In [20]:
#drop user table for test
sql = "DROP TABLE post"
mycursor.execute(sql)
sql = "DROP TABLE user"
mycursor.execute(sql)


In [24]:
#MongoDB
myclient = pymongo.MongoClient("mongodb://localhost:27017/")

In [25]:
#create database
mydbmongo = myclient["TwitterDB"]
#create collection
mycol = mydbmongo["Tweets"]

In [26]:
#start loading
#load function first

with open("AI-out", "r") as f1:
    for line in f1:
        
        try:
            data = json.loads(line)
            #print(data['id'])
            ProcessData(data) 
            
        except OSError as err:
            print("OS error: {0}".format(err))
        except ValueError:
            pass
        except:
            print("Unexpected error:", sys.exc_info()[0])
            raise


<ipython-input-17-58ba63afac64>:91: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  if mycol.find({'_id': tweet_id}).limit(1).count() > 0:


In [13]:
def ProcessData(data):
    
#-----------user infomation----------
    #tweet user
    if checkExist(data['user']["id"]):
        #already exist 
        updateUser(data['user'])
    else:
        #new user
        createNewUser(data['user']) 
        
        
    #retweet user
    if "retweeted_status" in data:
        if checkExist(data['retweeted_status']['user']["id"]):
            #already exist
            updateUser(data['retweeted_status']['user'])
        else:
            #new user
            createNewUser(data['retweeted_status']['user'])
            
#------------post ----------------
    newPost(data)
    
        
    

In [14]:
def checkExist(user_id):
    idquery= "SELECT COUNT(1) FROM user WHERE user_id = "+str(user_id)+";"
    mycursor.execute(idquery)
    myresult = mycursor.fetchall()

    if(myresult[0][0]!=0):
        return True
    else:
        return False
    

In [15]:
def createNewUser(data):
    user_id = data["id"]
    name = data["name"]
    screen_name=data["screen_name"]
    location = data["location"]
    description = data["description"]
    followers_count = data["followers_count"]
    friends_count = data["friends_count"]
    listed_count = data["listed_count"]
    favourites_count = data["favourites_count"]
    statuses_count = data["statuses_count"]
    
    new_datetime = datetime.strftime(datetime.strptime(data["created_at"],'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
    created_at = new_datetime
    
    sql = ("INSERT INTO user (user_id, name, screen_name, location,"+
    " description, followers_count, friends_count, listed_count, favourites_count, statuses_count, created_at)"+
    " VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")
    val=(user_id,name,screen_name,location,description,int(followers_count)
         ,int(friends_count),int(listed_count),int(favourites_count),int(statuses_count),created_at)
 
    mycursor.execute(sql, val)
    mydbsql.commit()

In [16]:
def updateUser(data):
    user_id = data["id"]
    name = data["name"]
    screen_name=data["screen_name"]
    location = data["location"]
    description = data["description"]
    followers_count = data["followers_count"]
    friends_count = data["friends_count"]
    listed_count = data["listed_count"]
    favourites_count = data["favourites_count"]
    statuses_count = data["statuses_count"]
    
    sql = ("UPDATE user SET name = %s, "+
    "screen_name= %s, "+
    "location= %s, "+
    "description= %s, "+
    "followers_count= %s, "+
    "friends_count= %s, "+
    "listed_count= %s, "+
    "favourites_count= %s, "+
    "statuses_count= %s "+
    "WHERE user_id = %s")
    
    
    val = (name,screen_name,location,description,int(followers_count),
           int(friends_count),int(listed_count),int(favourites_count),int(statuses_count),user_id)
    #print(sql)
    mycursor.execute(sql, val)
    mydbsql.commit()

In [17]:
def newPost(data):
    if "retweeted_status" in data:
        # it is a RT
        RT = 1
        
        #for tweet
        if "extended_tweet" in data:
            #for tweet mongoDB
            tweet_id = data['id']
            text_full = data['extended_tweet']['full_text']
            text=text_full.partition('RT')[0]
            
            hashtag = []
            if("extended_tweet" in data['retweeted_status']):
                for ht in (data['retweeted_status']['extended_tweet']['entities']['hashtags']):
                    hashtag.append(ht['text'])
            else:
                for ht in (data['retweeted_status']['entities']['hashtags']):
                    hashtag.append(ht['text'])
        
            new_datetime = datetime.strftime(datetime.strptime(data["created_at"],'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
            created_at = new_datetime
        
            quote_count=data['quote_count']
            reply_count=data['reply_count']
            retweet_count=data['retweet_count']
            favorite_count=data['favorite_count']
            
            user_id=data['user']['id']
            user_screen_name=data['user']['screen_name']
            
            rt_id=data['retweeted_status']['id']
            
        
            mydict = { "_id": tweet_id,"RT":RT,"text":text,"hashtag":hashtag,"created_at":created_at,
                      "quote_count":quote_count,"reply_count":reply_count,"retweet_count":retweet_count,
                      "favorite_count":favorite_count,"user_screen_name":user_screen_name,"rt_id":rt_id}
        
            x = mycol.insert_one(mydict)
        
            #for post table mySQL
        
            sql = ("INSERT INTO post (user_id, post_id, created_at) VALUES (%s, %s, %s)")
            val=(user_id,tweet_id,created_at)
        
            mycursor.execute(sql, val)
            mydbsql.commit()
        
        else:
            #for tweet mongoDB
            tweet_id = data['id']
            text_full = data['text']
            text=text_full.partition('RT')[0]
            
            hashtag = []
            if("extended_tweet" in data['retweeted_status']):
                for ht in (data['retweeted_status']['extended_tweet']['entities']['hashtags']):
                    hashtag.append(ht['text'])
            else:
                for ht in (data['retweeted_status']['entities']['hashtags']):
                    hashtag.append(ht['text'])
        
            new_datetime = datetime.strftime(datetime.strptime(data["created_at"],'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
            created_at = new_datetime
        
            quote_count=data['quote_count']
            reply_count=data['reply_count']
            retweet_count=data['retweet_count']
            favorite_count=data['favorite_count']
            
            user_screen_name=data['user']['screen_name']
            rt_id=data['retweeted_status']['id']
        
            mydict = { "_id": tweet_id,"RT":RT,"text":text,"hashtag":hashtag,"created_at":created_at,
                      "quote_count":quote_count,"reply_count":reply_count,"retweet_count":retweet_count,
                      "favorite_count":favorite_count,"user_screen_name":user_screen_name,"rt_id":rt_id}
        
            x = mycol.insert_one(mydict)
        
            #for post table mySQL
            user_id=data['user']['id']
        
            sql = ("INSERT INTO post (user_id, post_id, created_at) VALUES (%s, %s, %s)")
            val=(user_id,tweet_id,created_at)
        
            mycursor.execute(sql, val)
            mydbsql.commit()
        
        #for retweet
        tweet_id = data['retweeted_status']['id']
        if mycol.find({'_id': tweet_id}).limit(1).count() > 0:
            # existed retweet
            pass
        else:
            # new retweet
            #print(data['retweeted_status'])
            insertNewPost(data['retweeted_status'])
        
    else:
        # not a RT
        insertNewPost(data)
        

In [18]:
def insertNewPost(data):
    if "extended_tweet" in data:
        #for tweet mongoDB
        tweet_id = data['id']
        RT = 0
        text = data['extended_tweet']['full_text']
        hashtag = []
        for ht in (data['extended_tweet']['entities']['hashtags']):
            hashtag.append(ht['text'])
        
        new_datetime = datetime.strftime(datetime.strptime(data["created_at"],'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
        created_at = new_datetime
    
        quote_count=data['quote_count']
        reply_count=data['reply_count']
        retweet_count=data['retweet_count']
        favorite_count=data['favorite_count']
            
        user_id=data['user']['id']
        user_screen_name=data['user']['screen_name']
        
        mydict = { "_id": tweet_id,"RT":RT,"text":text,"hashtag":hashtag,"created_at":created_at,
                  "quote_count":quote_count,"reply_count":reply_count,"retweet_count":retweet_count,
                  "favorite_count":favorite_count,"user_screen_name":user_screen_name}
        
        x = mycol.insert_one(mydict)
        
        #for post table mySQL
        
        sql = ("INSERT INTO post (user_id, post_id, created_at) VALUES (%s, %s, %s)")
        val=(user_id,tweet_id,created_at)
    
        mycursor.execute(sql, val)
        mydbsql.commit()
        
    else:
        #for tweet mongoDB
        tweet_id = data['id']
        RT = 0
        text = data['text']
        hashtag = []
        for ht in (data['entities']['hashtags']):
            hashtag.append(ht['text'])
        
        new_datetime = datetime.strftime(datetime.strptime(data["created_at"],'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
        created_at = new_datetime
        
        quote_count=data['quote_count']
        reply_count=data['reply_count']
        retweet_count=data['retweet_count']
        favorite_count=data['favorite_count']
            
        user_screen_name=data['user']['screen_name']
        
        mydict = { "_id": tweet_id,"RT":RT,"text":text,"hashtag":hashtag,"created_at":created_at,
                  "quote_count":quote_count,"reply_count":reply_count,"retweet_count":retweet_count,
                  "favorite_count":favorite_count,"user_screen_name":user_screen_name}
        
        x = mycol.insert_one(mydict)
        
        #for post table mySQL
        user_id=data['user']['id']
        
        sql = ("INSERT INTO post (user_id, post_id, created_at) VALUES (%s, %s, %s)")
        val=(user_id,tweet_id,created_at)
        
        mycursor.execute(sql, val)
        mydbsql.commit()


# Search Engine

In [52]:
import pandas as pd
import time
import numpy as np
import re

In [50]:
def main():

In [32]:
total=[]
for i in range(100):
    t0 = time.time()
    res=[]

    myquery = { "user_screen_name": "experienceads" }
    mydoc = mycol.find(myquery)

    for y in mydoc:
        res.append(y) 
        
    t1 = time.time()
    total.append(t1-t0) 

np.average(total)
print(res)


[{'_id': 1380932241287692288, 'RT': 0, 'text': 'The latest Jimi Hendrix Daily News! https://t.co/ebDcoqixoc Thanks to @YvonneMcCulley1 @BMFDV #artificialintelligence', 'hashtag': ['artificialintelligence'], 'created_at': '2021-04-10 17:14:37', 'quote_count': 0, 'reply_count': 0, 'retweet_count': 0, 'favorite_count': 0, 'user_screen_name': 'experienceads'}]


In [107]:
total=[]
for i in range(100):
    res=[]
    t0 = time.time()
    sql = "SELECT * FROM post WHERE user_id= 10541332;"

    mycursor.execute(sql)
    myresult = mycursor.fetchall()

    for x in myresult:
        myquery = { "_id": int(x[1]) }
        mydoc = mycol.find(myquery)
        
        for y in mydoc:
            res.append(y)
            
        
    t1 = time.time()
    total.append(t1-t0) 

np.average(total)



0.006366369724273682

In [48]:
def searchUser(name):
    
    sql = "SELECT * FROM user WHERE (user_id = %s) OR (name = %s) OR (screen_name = %s);"
    val=(name,name,name)
    
    mycursor.execute(sql, val)

    myresult = mycursor.fetchall()

    
    x=myresult[0]
    df = pd.DataFrame([list(x),], columns =["id", "name", "screen_name", "location",
                                            "description","followers_count","friends_count",
                                            "listed_count","favourites_count","statuses_count","created_at"])
    print(df)
    
    t0 = time.time()
    
    user_id=x[0]
    sql = 
    
    
    t1 = time.time()
    total = t1-t0
    

In [58]:
def searchTag(hashtag):
    res = []
    
    myquery = {"hashtag": hashtag}
    mydoc = mycol.find(myquery).sort("retweet_count", -1)
    
    for y in mydoc:
        res.append(y)
    
    if res:
        return res
    else:
        print("No tweets relates to this hashtag")
    
searchTag("artificialintelligence")[:3]

[{'_id': 1207040017350242304,
  'RT': 0,
  'text': 'Artificial intelligence and automation are already causing friction in the workforce. Should schools revamp existing programs for topics like #AI, or are new research areas required? @CIOdive @its_ahickey #artificialintelligence #highereducation #college \nhttps://t.co/r2SRhz6Ymp',
  'hashtag': ['AI', 'artificialintelligence', 'highereducation', 'college'],
  'created_at': '2019-12-17 20:49:00',
  'quote_count': 1,
  'reply_count': 0,
  'retweet_count': 180,
  'favorite_count': 34,
  'user_screen_name': 'axelrod_eric'},
 {'_id': 1207030957938814976,
  'RT': 0,
  'text': '#artificialintelligence and #automation are already causing friction in the workforce. Should schools revamp existing programs for topics like #AI, or are new research areas required? @CIOdive @its_ahickey #highereducation #college #university\nhttps://t.co/JQnLaOhnWC',
  'hashtag': ['artificialintelligence',
   'automation',
   'AI',
   'highereducation',
   'college

In [106]:
def validate(date_text):
    try:
        datetime.strptime(date_text, "%Y-%m-%d")
    except ValueError:
        raise ValueError("Incorrect data format, should be YYYY-MM-DD")
        
def searchtime(beginning, ending):
    res = []
    
    #validate if user input correct format of date
    validate(beginning)
    validate(ending)
    
    beginning = beginning + str(" 00:00:00")
    ending = ending + str(" 23:59:59")
    myquery = {"created_at": {
        "$gte": beginning,
        "$lte": ending
    } 
    }
    mydoc = mycol.find(myquery).sort("created_at", -1)
    
    for i in mydoc:
        res.append(i)
    
    return res

#new_datetime = datetime.strftime(datetime.strptime(data["created_at"],'%a %b %d %H:%M:%S +0000 %Y'), '%Y-%m-%d %H:%M:%S')
#created_at = new_datetime

In [107]:
searchtime("2019-12-16", "2021-01-01")

[{'_id': 1344395301935849473,
  'RT': 0,
  'text': '#artificialintelligence and #automation are already causing friction in the workforce. Should schools revamp existing programs for topics like #AI, or are new research areas required? @CIOdive @its_ahickey #highereducation #college #university #ai\nhttps://t.co/y3OppT9jle',
  'hashtag': ['artificialintelligence',
   'automation',
   'AI',
   'highereducation',
   'college',
   'university',
   'ai'],
  'created_at': '2020-12-30 21:29:52',
  'quote_count': 0,
  'reply_count': 0,
  'retweet_count': 100,
  'favorite_count': 14,
  'user_screen_name': 'digr_io'},
 {'_id': 1343519651985240064,
  'RT': 0,
  'text': "5⭐️...elements of thrilling mystery, suspense, dystopian fantasy and science fiction.'\n\nThe Lenders Saga from @Travis_Borne.\n\n#scifi #sciencefiction #AI #ArtificialIntelligence #Aliens #timetravel #apocalypse #IARTG #ASMSG \n#Kindle #books #ebooks \nhttps://t.co/kqLbqCb1g9 https://t.co/pfeoSDXTqI",
  'hashtag': ['scifi',
   '

In [49]:
def search():
    while True:
        statement = input("For user search, enter 1\n"+"For tweet search, enter 2\n"+
                          "For hashtag search, enter 3\n"+"For time range search, enter 4\n"+
                          "To exit, enter 0\n")
        if(statement=='0'):
            print("Exit")
            break
        elif(statement=='1'):
            statement = input("Please enter the user name, screen name, or id:\n")
            searchUser(statement)
            break
        elif(statement=='2'):
            break
        elif(statement=='3'):
            break
        elif(statement=='4'):
            break
        else:
            print("invalid input")

In [50]:
search()
    

For user search, enter 1
For tweet search, enter 2
For hashtag search, enter 3
For time range search, enter 4
To exit, enter 0
1
Please enter the user name, screen name, or id:
experienceads
         id     name    screen_name     location  \
0  10541332  Evan W.  experienceads  Sunrise, FL   

                                         description  followers_count  \
0  #InternetMarketing enthusiast at Experience Ad...            11895   

   friends_count  listed_count  favourites_count  statuses_count  \
0          13197           637              7185           30353   

           created_at  
0 2007-11-25 02:13:29  


In [ ]:
if __name__ == "__main__":
    main()
    
    

# Caching

In [ ]:
class Node(object):
    def __init__(self, key, value, next, prev):
        self.value = value
        self.key = key
        self.after = after
        self.prev = prev

In [ ]:
class DoublyLinkedList(object):
    def __init__(self):
        self.head = None
        self.tail = None
        self.__size = 0
        
    # insert at beginning of linkedlist
    def append(self, node):
        if self.head == None:
            new_Node = Node(node)
            self.head = new_Node
        else:
            new_Node = Node(node)
            self.head.prev = new_Node
            new_Node.next = self.head
            self.head = new_Node

    #remove and return the node we want 
    def remove(self, node):
        if self.head == node:
            self.head = self.head.next
        if self.tail == node:
            self.tail = self.tail.prev

        next_node = node.next
        prev_node = node.prev

        if prev_node:
            prev_node.next = next_node
        if next_node:
            next_node.prev = prev_node

        self.__size -= 1
        return node
    
    #remove and return last node
    def pop(self):
        tail_node = self.tail
        prev_node = tail_node.prev
        
        if prev_node:
            prev_node.next = None
            
        self.tail = prev_node
        
        self.__size -= 1
        
        return tail_node
    
    def __len__(self):
        return self.__size

In [ ]:
class Cache:

    def __init__(self, capacity):
        self._capacity = capacity
        self._lookup = {}
        self._data = DoublyLinkedList()

    def set(self, key, value):
        if key in self._lookup: 
            current_node = self._lookup[key]
            current_node.value = value

            self._data.remove(current_node)
            self._data.append(current_node)
        else:
            new_node = Node(key, value)
            self._lookup[key] = new_node
            self._data.append(new_node)
        
        # if larger than capacity 
        if len(self._lookup) > self._capacity:
            stale_node = self._data.pop()
            del self._lookup[stale_node.key]


        def get(self, key):
            if key not in self._lookup: 
                return 
            current_node = self._lookup[key]
            self._data.remove(current_node)
            self._data.append(current_node)
            return current_node.value


    def __repr__(self):
        return self._data.__repr__()

In [63]:
myclient.close()

In [64]:
mycursor.close()

True